In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
# Step 1: Load and Parse Activity Labels
print("Loading and parsing activity labels...")
# Step 1: Load and Parse Activity Labels
print("Loading and parsing activity labels...")
activities_df = pd.read_csv(r'C:\projs\tmc\TrainingDataPD25\TrainingDataPD25\TrainActivities.csv')

Loading and parsing activity labels...
Loading and parsing activity labels...


### dateTime-transform

In [19]:
date_columns = ['Started', 'Finished', 'Updated']

# Convert while keeping NaNs intact
activities_df[date_columns] = activities_df[date_columns].apply(
    pd.to_datetime, format='%Y/%m/%d %H:%M', errors='coerce'
)

# Check result
print(activities_df[date_columns].dtypes)
activities_df.head()


Started     datetime64[ns]
Finished    datetime64[ns]
Updated     datetime64[ns]
dtype: object


,ID,Activity Type ID,Activity Type,Started,Finished,Updated,Subject
0,1130251,2806,1 (FACING camera) Sit and stand,2024-09-02 06:16:00,2024-09-02 06:16:00,2024-09-02 06:16:00,U22
1,1130254,2807,2 (FACING camera) both hands SHAKING (sitting ...,2024-09-02 06:17:00,2024-09-02 06:17:00,2024-09-02 06:17:00,U22
2,1130257,2807,2 (FACING camera) both hands SHAKING (sitting ...,2024-09-02 06:18:00,2024-09-02 06:18:00,2024-09-02 06:18:00,U22
3,1130261,2806,1 (FACING camera) Sit and stand,2024-09-02 06:20:00,2024-09-02 06:20:00,2024-09-02 06:20:00,U22
4,1130292,2806,1 (FACING camera) Sit and stand,2024-09-02 06:42:00,2024-09-02 06:42:00,2024-09-02 06:42:00,U2


### Checking for NaNs

In [22]:
# List the columns you want to check
columns_to_check = activities_df.columns  # change this list as needed

# Check for NaN/NaT counts in each of the specified columns
nan_counts = activities_df[columns_to_check].isna().sum()

# Display the result
print("NaN/NaT counts per column:")
print(nan_counts)


NaN/NaT counts per column:
ID                   0
Activity Type ID     0
Activity Type        0
Started             66
Finished            67
Updated              0
Subject              0
dtype: int64


In [24]:
activities_df

,ID,Activity Type ID,Activity Type,Started,Finished,Updated,Subject
0,1130251,2806,1 (FACING camera) Sit and stand,2024-09-02 06:16:00,2024-09-02 06:16:00,2024-09-02 06:16:00,U22
1,1130254,2807,2 (FACING camera) both hands SHAKING (sitting ...,2024-09-02 06:17:00,2024-09-02 06:17:00,2024-09-02 06:17:00,U22
2,1130257,2807,2 (FACING camera) both hands SHAKING (sitting ...,2024-09-02 06:18:00,2024-09-02 06:18:00,2024-09-02 06:18:00,U22
3,1130261,2806,1 (FACING camera) Sit and stand,2024-09-02 06:20:00,2024-09-02 06:20:00,2024-09-02 06:20:00,U22
4,1130292,2806,1 (FACING camera) Sit and stand,2024-09-02 06:42:00,2024-09-02 06:42:00,2024-09-02 06:42:00,U2
...,...,...,...,...,...,...,...
337,1164181,2815,"10 Slow walk (SHAKING hands/body, tiny step, h...",NaT,NaT,2024-09-11 04:55:00,U7
338,1164187,2806,1 (FACING camera) Sit and stand,2024-09-11 04:58:00,2024-09-11 04:58:00,2024-09-11 04:58:00,U7
339,1164217,2815,"10 Slow walk (SHAKING hands/body, tiny step, h...",2024-09-11 05:09:00,2024-09-11 05:10:00,2024-09-11 05:10:00,U7
340,1164218,2815,"10 Slow walk (SHAKING hands/body, tiny step, h...",2024-09-11 05:10:00,2024-09-11 05:10:00,2024-09-11 05:10:00,U7


,ID,Activity Type ID,Activity Type,Started,Finished,Updated,Subject
176,1144326,2806,1 (FACING camera) Sit and stand,NaT,NaT,2024-09-05 17:33:00,U3
177,1144327,2806,1 (FACING camera) Sit and stand,NaT,NaT,2024-09-05 17:33:00,U3
178,1144328,2807,2 (FACING camera) both hands SHAKING (sitting ...,NaT,NaT,2024-09-05 17:33:00,U3
179,1144329,2807,2 (FACING camera) both hands SHAKING (sitting ...,NaT,NaT,2024-09-05 17:33:00,U3
180,1144330,2807,2 (FACING camera) both hands SHAKING (sitting ...,NaT,NaT,2024-09-05 17:33:00,U3
...,...,...,...,...,...,...,...
251,1160418,2807,2 (FACING camera) both hands SHAKING (sitting ...,NaT,NaT,2024-09-10 05:49:00,U6
280,1160468,2810,5 (Sideway) both hands SHAKING (sitting),NaT,NaT,2024-09-10 06:08:00,U6
335,1164179,2815,"10 Slow walk (SHAKING hands/body, tiny step, h...",NaT,NaT,2024-09-11 04:55:00,U7
336,1164180,2815,"10 Slow walk (SHAKING hands/body, tiny step, h...",NaT,NaT,2024-09-11 04:55:00,U7


In [ ]:
# Parse datetime and localize to UTC
activities_df['Started'] = pd.to_datetime(activities_df['Started'], format='mixed', dayfirst=True)
activities_df['Finished'] = pd.to_datetime(activities_df['Finished'], format='mixed', dayfirst=True)

# Localize to UTC (if tz-naive)
if activities_df['Started'].dt.tz is None:
    activities_df['Started'] = activities_df['Started'].dt.tz_localize('UTC')
else:
    activities_df['Started'] = activities_df['Started'].dt.tz_convert('UTC')

if activities_df['Finished'].dt.tz is None:
    activities_df['Finished'] = activities_df['Finished'].dt.tz_localize('UTC')
else:
    activities_df['Finished'] = activities_df['Finished'].dt.tz_convert('UTC')

activities_df['Activity_Label'] = activities_df['Activity Type'].str.extract(r'^(\d+)').astype(int)

# Group activities by subject
subject_activities = activities_df.groupby('Subject')
print(f"Found activities for {len(subject_activities)} subjects.")

In [11]:
def get_time_range(subdir):
    """Get the min and max timestamps from all CSV files in a subdirectory."""
    csv_files = [f for f in os.listdir(os.path.join(data_dir, subdir)) if f.endswith('.csv')]
    
    min_ts, max_ts = None, None
    for csv_file in csv_files:
        file_path = os.path.join(data_dir, subdir, csv_file)
        df = pd.read_csv(file_path, usecols=[1], header=None)
        df.columns = ['Timestamp']
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='mixed', dayfirst=True)

        # Convert to UTC
        df['Timestamp'] = df['Timestamp'].dt.tz_localize('UTC') if df['Timestamp'].dt.tz is None else df['Timestamp'].dt.tz_convert('UTC')

        current_min = df['Timestamp'].min()
        current_max = df['Timestamp'].max()
        min_ts = current_min if min_ts is None or current_min < min_ts else min_ts
        max_ts = current_max if max_ts is None or current_max > max_ts else max_ts
    return min_ts, max_ts

# Compute time ranges for each subdirectory
subdir_time_ranges = {subdir: get_time_range(subdir) for subdir in subdirs}

def intervals_overlap(subdir_min, subdir_max, intervals):
    """Check if subdirectory time range overlaps with any activity interval."""
    
    # Localize subdir timestamps if needed
    if subdir_min.tzinfo is None:
        subdir_min = subdir_min.tz_localize('UTC')
    else:
        subdir_min = subdir_min.tz_convert('UTC')

    if subdir_max.tzinfo is None:
        subdir_max = subdir_max.tz_localize('UTC')
    else:
        subdir_max = subdir_max.tz_convert('UTC')

    for start, end in intervals:
        # Localize each interval if needed
        if start.tzinfo is None:
            start = start.tz_localize('UTC')
        else:
            start = start.tz_convert('UTC')

        if end.tzinfo is None:
            end = end.tz_localize('UTC')
        else:
            end = end.tz_convert('UTC')

        if subdir_min <= end and subdir_max >= start:
            return True

    return False


# Map subdirectories to subjects based on time range overlap
subdir_to_subject = {}
for subdir, (subdir_min, subdir_max) in subdir_time_ranges.items():
    matching_subjects = [
        subject for subject, group in subject_activities
        if intervals_overlap(subdir_min, subdir_max, list(zip(group['Started'], group['Finished'])))
    ]
    if len(matching_subjects) == 1:
        subdir_to_subject[subdir] = matching_subjects[0]
    else:
        print(f"Warning: Subdirectory {subdir} matches {len(matching_subjects)} subjects.")

print(f"Mapped {len(subdir_to_subject)} subdirectories to subjects.")

# Step 3: Load and Label Accelerometer Data
all_labeled_data = []
for subdir, subject in subdir_to_subject.items():
    csv_files = [f for f in os.listdir(os.path.join(data_dir, subdir)) if f.endswith('.csv')]
    data_list = []
    for csv_file in csv_files:
        file_path = os.path.join(data_dir, subdir, csv_file)
        # Assuming columns: random (0), timestamp (1), x (2), y (3), z (4)
        df = pd.read_csv(file_path, usecols=[1, 2, 3, 4], header=None)
        df.columns = ['Timestamp', 'x', 'y', 'z']
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        data_list.append(df)
    subdir_data = pd.concat(data_list, ignore_index=True)
    subdir_data['Subject'] = subject
    
    # Label data points with activity labels
    activities = subject_activities.get_group(subject)
    for _, activity in activities.iterrows():
        start = activity['Started']
        end = activity['Finished']
        activity_label = activity['Activity_Label']
        mask = (subdir_data['Timestamp'] >= start) & (subdir_data['Timestamp'] <= end)
        subdir_data.loc[mask, 'Activity'] = activity_label
    
    # Keep only labeled data
    labeled_data = subdir_data.dropna(subset=['Activity'])
    all_labeled_data.append(labeled_data)
    print(f"Processed data for subject {subject} from subdirectory {subdir}.")

all_data = pd.concat(all_labeled_data, ignore_index=True)
print(f"Total labeled data points: {len(all_data)}")

Mapped 1 subdirectories to subjects.
Processed data for subject U7 from subdirectory 2812.
Total labeled data points: 0


In [15]:
subdir_to_subject

{'2812': 'U7'}

In [13]:
# Step 4: Segment Data into Windows
def segment_into_windows(df, window_size, stride):
    """Segment data into fixed-size windows with specified stride."""
    df = df.sort_values(by=['Subject', 'Timestamp'])
    df['activity_change'] = df['Activity'].ne(df['Activity'].shift()).cumsum()
    grouped = df.groupby(['Subject', 'activity_change'])
    windows = []
    labels = []
    for _, group in grouped:
        group = group.sort_values('Timestamp')
        num_samples = len(group)
        for start in range(0, num_samples - window_size + 1, stride):
            end = start + window_size
            window = group.iloc[start:end]
            if len(window) == window_size:
                windows.append(window[['x', 'y', 'z']].values)
                labels.append(window['Activity'].iloc[0] - 1)  # Adjust to 0-based indexing
    return np.array(windows), np.array(labels)

# Define training and validation subjects
train_subjects = ['U1', 'U2', 'U3', 'U4', 'U5', 'U6', 'U7']
val_subjects = ['U21', 'U22']

train_data = all_data[all_data['Subject'].isin(train_subjects)]
val_data = all_data[all_data['Subject'].isin(val_subjects)]

window_size = 100  # e.g., 2 seconds at 50Hz
stride = 50       # e.g., 1-second overlap
print("Segmenting data into windows...")
train_windows, train_labels = segment_into_windows(train_data, window_size, stride)
val_windows, val_labels = segment_into_windows(val_data, window_size, stride)
print(f"Training windows: {len(train_windows)}, Validation windows: {len(val_windows)}")

# Step 5: Build and Train the CNN Model
model = Sequential([
    Conv1D(32, kernel_size=5, activation='relu', input_shape=(window_size, 3)),
    MaxPooling1D(pool_size=2),
    Conv1D(64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

print("Training the model...")
model.fit(
    train_windows, train_labels,
    epochs=50,
    batch_size=32,
    validation_data=(val_windows, val_labels),
    verbose=1
)

print("Model training completed.")

Segmenting data into windows...
Training windows: 0, Validation windows: 0


NameError: name 'Sequential' is not defined